In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 71kB/s 
     |████████████████████████████████| 204kB 34.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=8f9dc8344fd31c197a20a3119ac97cde6a0ed433acc0c79d5328a4f3c5a5c5db
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


***Przewidywanie wzięcia pożyczki. ***

Celem projektu jest zastosowanie modeli  klasyfikacji (binarnej) do  przewidzenia czy pożyczka zostanie udzielona danemu klientowi z danymi parametrami wniosku. 

Zmienna celu (wypłacenie pożyczki): ***Disbursed ***


---



***Wymogi:*** 

● Zmienna LoggedIn nie może być uwzględniona w modelowaniu - należy ją od razu wyrzucić ze  zbioru.  
● Nie usuwamy żadnych obserwacji - braki danych uzupełniamy w jakiś sposób.  
● Wszelkie nieoczywiste ​ przekształcenia danych należy opatrzyć uzasadnieniem​ dlaczego  dokonujemy tego przekształcenia (np. robimy wykres i uzasadniamy wzięci logarytmu ze  zmiennej tym, że rozkład jest skośny).  
● Należy wykorzystać przynajmniej 3 metody klasyfikacji.  
● Należy uwzględnić optymalizację modeli/pipelinów.  
● Projekt musi być zakończony przejrzystym porównaniem przetestowanych rozwiązań  (modeli/pipelinów) ​ w postaci tabeli​ , gdzie jeden wiersz opisuje jedno rozwiązanie i jego  wynik. 



Link do danych:  https://raw.githubusercontent.com/saimadhu-polamuri/DataHakthon3X/master/dataSet/Train.csv 


Opis danych (najdokładniejszy jaki istnieje):  https://discuss.analyticsvidhya.com/t/hackathon-3-x-predict-customer-worth-for-happy-customer-bank/3802 

**IMPORT BIBLIOTEK**

In [5]:
import pyspark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve



In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName('my_app').getOrCreate()

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import feature

In [8]:
spark

***Import Danych, Przekształcenia***

In [9]:
csv_df = spark.read.csv('/content/Train.csv', header=True, inferSchema=True)


In [10]:
csv_df.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+--------+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|LoggedIn|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------

Wyrzucenie ze zbioru zmiennej **LoggedIn**

In [14]:
tmp = csv_df.drop('LoggedIn')
tmp.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+

In [15]:
tmp.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = true)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = true)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = true)
 |-- Filled_Form: string (nullable = true)
 |-- Device_Type: string (nullable = true)
 |-- Var2: string (nullable = true)
 |-- Source: string (nul

In [16]:
tmp.describe().show()

+-------+-----------+------+--------------+------------------+---------+------------------+-------------------+-------------------+-----------------+--------------------+--------------------+---------------+-----------------+-----+---------------------+---------------------+------------------+------------------+------------------+-----------+-----------+-----+------+------------------+-------------------+
|summary|         ID|Gender|          City|    Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|     Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|             Var5| Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|     Interest_Rate|    Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type| Var2|Source|              Var4|          Disbursed|
+-------+-----------+------+--------------+------------------+---------+------------------+-------------------+-------------------+-----------------+--------------------+------------

In [17]:
# zmieniamy Null na 0

tmp = tmp.fillna(0)
tmp.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+

In [18]:
tmp.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = false)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = false)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = false)
 |-- Filled_Form: string (nullable = true)
 |-- Device_Type: string (nullable = true)
 |-- Var2: string (nullable = true)
 |-- Source: string (

In [19]:
tmp.groupBy("Disbursed").count().show()

+---------+-----+
|Disbursed|count|
+---------+-----+
|        1| 1273|
|        0|85747|
+---------+-----+



In [ ]:
#tmp.toPandas()


In [20]:
# Liczba pożyczek
tmp.select((tmp["Disbursed"]).alias("cln")).describe().show()

+-------+-------------------+
|summary|                cln|
+-------+-------------------+
|  count|              87020|
|   mean|0.01462882096069869|
| stddev|0.12006241797245076|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [21]:
tmp.createOrReplaceTempView("tmp")

In [22]:
# max/min/avg udzielonej pożyczki

spark.sql('select max(Loan_Amount_Applied) as Max_Loan_Amount, min(Loan_Amount_Applied) as Min_Loan_Amount, avg(Loan_Amount_Applied) as Avg_Loan_Amout \
      from tmp \
      where Disbursed == 1').show()

+---------------+---------------+-----------------+
|Max_Loan_Amount|Min_Loan_Amount|   Avg_Loan_Amout|
+---------------+---------------+-----------------+
|        3000000|              0|325259.2482325216|
+---------------+---------------+-----------------+



In [23]:
# sprawdzenie unikalności danych
tmp.groupBy("Device_Type").count().show()
tmp.groupBy("Gender").count().show()
tmp.groupBy("Filled_Form").count().show()
tmp.groupBy("Source").count().show()
tmp.groupBy("Mobile_Verified").count().show()

+-----------+-----+
|Device_Type|count|
+-----------+-----+
|Web-browser|64316|
|     Mobile|22704|
+-----------+-----+

+------+-----+
|Gender|count|
+------+-----+
|Female|37172|
|  Male|49848|
+------+-----+

+-----------+-----+
|Filled_Form|count|
+-----------+-----+
|          Y|19490|
|          N|67530|
+-----------+-----+

+------+-----+
|Source|count|
+------+-----+
|  S155|    4|
|  S143| 4332|
|  S158|  208|
|  S151|  720|
|  S162|   36|
|  S136|    3|
|  S157|  650|
|  S160|   11|
|  S159| 5599|
|  S161|  769|
|  S129|    3|
|  S135|    2|
|  S125|    1|
|  S156|  308|
|  S154|    1|
|  S150|   10|
|  S138|    3|
|  S122|38567|
|  S133|29885|
|  S123|   73|
+------+-----+
only showing top 20 rows

+---------------+-----+
|Mobile_Verified|count|
+---------------+-----+
|              Y|56481|
|              N|30539|
+---------------+-----+



Konwersja danych

In [24]:
from pyspark.sql.functions import col, when

In [25]:
tmp1 = tmp.select(col("*"), when(col("Gender") == "Male",1)
      .when(col("Gender") == "Female",0).alias("new_gender"), when(col("Filled_Form") == "Y",1)
      .when(col("Filled_Form") == "N",0).alias("new_Filled_Form"), when(col("Device_Type") == "Web-browser",1)
      .when(col("Device_Type") == "Mobile",0).alias("new_Device_Type"), when(col("Mobile_Verified") == "Y",1)
      .when(col("Mobile_Verified") == "N",0).alias("new_Mobile_Verified"))
tmp1.show()

+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+-----------+-----------+----+------+----+---------+----------+---------------+---------------+-------------------+
|         ID|Gender|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Filled_Form|Device_Type|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+------+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+

In [26]:
tmp2 = tmp1.drop("Gender", "Mobile_Veified", "Device_Type", "ID", "Filled_Form")
tmp2.show()


+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+---

In [27]:
tmp2.select("Employer_Name", "Disbursed").groupBy("Employer_Name").sum().orderBy(f.desc("sum(Disbursed)")).show()

+--------------------+--------------+
|       Employer_Name|sum(Disbursed)|
+--------------------+--------------+
|TATA CONSULTANCY ...|            21|
|ACCENTURE SERVICE...|            12|
|COGNIZANT TECHNOL...|            10|
|      INDIAN RAILWAY|             7|
|HCL TECHNOLOGIES LTD|             7|
|             GENPACT|             7|
|           CIPLA LTD|             5|
|              OTHERS|             5|
|  HCL AVITAS PVT LTD|             5|
|IBM GLOBAL SERVIC...|             5|
|TCS AND ASSOCIATE...|             5|
|                   0|             5|
|     IBM CORPORATION|             4|
|ERICSSON INDIA GL...|             4|
|                BSNL|             4|
|HCL - HEWLETT-PAC...|             4|
|CAPGEMINI INDIA P...|             4|
|   IDEA CELLULAR LTD|             4|
|TYPE SLOWLY FOR A...|             4|
|HSBC ELECTRONIC D...|             4|
+--------------------+--------------+
only showing top 20 rows



In [28]:
tmp2 = tmp2.fillna(0)
tmp2.show()

+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|       City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+---

In [29]:
df_t, df_e = tmp2.randomSplit([0.7, 0.3], 42)

In [30]:
df_t.show()

+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+-------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+
|    City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|     Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|
+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+-------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---

In [31]:
tmp2.printSchema()

root
 |-- City: string (nullable = true)
 |-- Monthly_Income: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Lead_Creation_Date: string (nullable = true)
 |-- Loan_Amount_Applied: integer (nullable = true)
 |-- Loan_Tenure_Applied: integer (nullable = true)
 |-- Existing_EMI: double (nullable = false)
 |-- Employer_Name: string (nullable = true)
 |-- Salary_Account: string (nullable = true)
 |-- Mobile_Verified: string (nullable = true)
 |-- Var5: integer (nullable = true)
 |-- Var1: string (nullable = true)
 |-- Loan_Amount_Submitted: integer (nullable = true)
 |-- Loan_Tenure_Submitted: integer (nullable = true)
 |-- Interest_Rate: double (nullable = false)
 |-- Processing_Fee: integer (nullable = true)
 |-- EMI_Loan_Submitted: double (nullable = false)
 |-- Var2: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Var4: integer (nullable = true)
 |-- Disbursed: integer (nullable = true)
 |-- new_gender: integer (nullable = true)
 |-- new_Filled_For

In [32]:
tmp2.show(3, vertical=True)

-RECORD 0-------------------------------------
 City                  | Delhi                
 Monthly_Income        | 20000                
 DOB                   | 23-May-78            
 Lead_Creation_Date    | 15-May-15            
 Loan_Amount_Applied   | 300000               
 Loan_Tenure_Applied   | 5                    
 Existing_EMI          | 0.0                  
 Employer_Name         | CYBOSOL              
 Salary_Account        | HDFC Bank            
 Mobile_Verified       | N                    
 Var5                  | 0                    
 Var1                  | HBXX                 
 Loan_Amount_Submitted | 0                    
 Loan_Tenure_Submitted | 0                    
 Interest_Rate         | 0.0                  
 Processing_Fee        | 0                    
 EMI_Loan_Submitted    | 0.0                  
 Var2                  | G                    
 Source                | S122                 
 Var4                  | 1                    
 Disbursed   

In [33]:
formula = ("Disbursed ~ new_gender + new_Device_Type + new_Mobile_Verified + Var1 + new_Filled_Form + Var2 + Source +"
                              "Monthly_Income + Loan_Amount_Applied + Loan_Tenure_Applied")
rf = feature.RFormula(formula=formula, featuresCol='featuresFull')

In [34]:
rfModel = rf.fit(tmp2)

In [35]:
tmp3 = rfModel.transform(tmp2)
tmp3.show(truncate=False)

+-----------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------------------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+-----------------------------------------------------------------------------+-----+
|City       |Monthly_Income|DOB      |Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|Employer_Name                               |Salary_Account      |Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|featuresFull                                                                 |label|
+-----------+--------------+---------+------------------+-----

In [36]:
#Skalowanie
scaler = feature.StandardScaler(inputCol="featuresFull", outputCol="featuresScal")

In [37]:
scalerModel = scaler.fit(tmp3)

In [38]:
tmp3.select("featuresFull").show(truncate=False)
scalerModel.transform(tmp3).select("featuresScal").show(truncate=False)

+-----------------------------------------------------------------------------+
|featuresFull                                                                 |
+-----------------------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |
|(60,[0,1,2,7,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|
|(60,[0,1,2,3,22,34,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,100000.0,500000.0,2.0])|
|(60,[0,1,2,15,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,45000.0,300000.0,5.0])|
|(60,[1,3,22,29,57,58,59],[1.0,1.0,1.0,1.0,70000.0,6.0,5.0])                  |
|(60,[0,1,2,3,22,30,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,20000.0,200000.0,5.0]) |
|(60,[0,2,8,21,24,28,57],[1.0,1.0,1.0,1.0,1.0,1.0,75000.0])                   |
|(60,[1,2,4,22,29,57,58,59],[1.0,1.0,1.0

In [39]:
MMscaler = feature.MinMaxScaler(inputCol="featuresFull", outputCol="featuresScal")

In [40]:
MMscalerModel = MMscaler.fit(tmp3)

In [41]:
tmp3.select("featuresFull").show(truncate=False)
MMscalerModel.transform(tmp3).select("featuresScal").show(truncate=False)

+-----------------------------------------------------------------------------+
|featuresFull                                                                 |
+-----------------------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |
|(60,[0,1,2,7,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|
|(60,[0,1,2,3,22,34,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,100000.0,500000.0,2.0])|
|(60,[0,1,2,15,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,45000.0,300000.0,5.0])|
|(60,[1,3,22,29,57,58,59],[1.0,1.0,1.0,1.0,70000.0,6.0,5.0])                  |
|(60,[0,1,2,3,22,30,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,20000.0,200000.0,5.0]) |
|(60,[0,2,8,21,24,28,57],[1.0,1.0,1.0,1.0,1.0,1.0,75000.0])                   |
|(60,[1,2,4,22,29,57,58,59],[1.0,1.0,1.0

**PCA**

In [42]:
pca = feature.PCA(k=3, inputCol="featuresFull", outputCol="featuresPCA")

In [43]:
PCAmodel = pca.fit(tmp3)

In [44]:
PCAmodel.transform(tmp3).select("featuresFull", "featuresPCA").show(truncate=False)

+-----------------------------------------------------------------------------+------------------------------------------------------------+
|featuresFull                                                                 |featuresPCA                                                 |
+-----------------------------------------------------------------------------+------------------------------------------------------------+
|(60,[1,3,23,28,57,58,59],[1.0,1.0,1.0,1.0,20000.0,300000.0,5.0])             |[-20152.98625505873,-299989.76174707466,4.097771557942352]  |
|(60,[0,1,2,7,23,28,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,200000.0,2.0]) |[-35101.98801937641,-199982.12529907905,1.3787087613486313] |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,22500.0,600000.0,4.0]) |[-22805.974785690658,-599988.4478268801,2.339282178163524]  |
|(60,[0,1,2,3,22,31,57,58,59],[1.0,1.0,1.0,1.0,1.0,1.0,35000.0,1000000.0,5.0])|[-35509.95830123298,-999982.0212799263,2.14316066116888]    |
|(60,[0,1,2,3

In [45]:
PCAmodel.explainedVariance

DenseVector([0.9742, 0.0258, 0.0])

In [46]:
rfModel = rf.fit(tmp2)

In [47]:
df_train = rfModel.transform(df_t)
df_eval = rfModel.transform(df_e)

In [48]:
scaler = feature.StandardScaler(inputCol="featuresFull", outputCol="features")
scal_mod = scaler.fit(df_train)
df_train = scal_mod.transform(df_train)
df_eval = scal_mod.transform(df_eval)

df_eval.show()

+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+---------------------+---------------------+-------------+--------------+------------------+----+------+----+---------+----------+---------------+---------------+-------------------+--------------------+-----+--------------------+
|    City|Monthly_Income|      DOB|Lead_Creation_Date|Loan_Amount_Applied|Loan_Tenure_Applied|Existing_EMI|       Employer_Name|      Salary_Account|Mobile_Verified|Var5|Var1|Loan_Amount_Submitted|Loan_Tenure_Submitted|Interest_Rate|Processing_Fee|EMI_Loan_Submitted|Var2|Source|Var4|Disbursed|new_gender|new_Filled_Form|new_Device_Type|new_Mobile_Verified|        featuresFull|label|            features|
+--------+--------------+---------+------------------+-------------------+-------------------+------------+--------------------+--------------------+---------------+----+----+-------------

In [49]:
num_cols = [c for c, t in df_t.dtypes if t != "string"]
categ_cols = [c for c, t in df_t.dtypes if t == "string" and c != "Disbursed"]
categ_cols_idx = [c + "Idx" for c in categ_cols]
categ_cols_vect = [c + "Vect" for c in categ_cols]

In [50]:
indexer = feature.StringIndexer(inputCol="Disbursed", outputCol="label1")
indexers = feature.StringIndexer(inputCols=categ_cols, outputCols=categ_cols_idx)
OHencoder = feature.OneHotEncoder(inputCols=categ_cols_idx, outputCols=categ_cols_vect)
vectAssembler = feature.VectorAssembler(inputCols = num_cols + categ_cols_vect, outputCol = "featuresRaw")
#scaler = feature.StandardScaler(inputCol="featuresRaw", outputCol="features")

In [51]:
pipe = Pipeline(stages=[indexer, indexers, OHencoder, vectAssembler, scaler])
pipeModel = pipe.fit(tmp3)

In [52]:
df_train = pipeModel.transform(tmp3)
df_eval = pipeModel.transform(tmp3)

In [ ]:
df_eval.show()

In [54]:
df_train = df_train.select("label", "features")
df_eval = df_eval.select("label", "features")

In [ ]:
print("Train:")
df_train.groupBy("label").count().show()
print("Eval:")
df_eval.groupBy("label").count().show()

In [56]:
df_train.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(60,[1,3,23,28,57...|
|  0.0|(60,[0,1,2,7,23,2...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,15,22,...|
|  0.0|(60,[1,3,22,29,57...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,2,8,21,24,...|
|  0.0|(60,[1,2,4,22,29,...|
|  0.0|(60,[0,1,2,4,22,2...|
|  0.0|(60,[1,3,22,28,57...|
|  0.0|(60,[1,3,22,29,57...|
|  0.0|(60,[0,1,2,9,22,2...|
|  0.0|(60,[1,3,22,36,57...|
|  0.0|(60,[0,1,2,3,22,3...|
|  0.0|(60,[0,1,2,11,25,...|
|  0.0|(60,[1,3,22,28,57...|
|  0.0|(60,[1,3,25,29,57...|
|  0.0|(60,[0,1,2,3,25,2...|
+-----+--------------------+
only showing top 20 rows



REGRESJA LINIOWA

In [70]:
from pyspark.ml import regression

In [71]:
reg = regression.LinearRegression(maxIter=500)

In [72]:
regModel = reg.fit(df_train)

In [73]:
regModel.coefficients

DenseVector([0.0031, 0.0, 0.0019, 0.0118, 0.0041, 0.002, 0.0047, 0.0075, 0.0089, 0.0049, 0.0081, 0.0015, 0.0028, 0.0012, 0.0016, 0.0023, 0.0007, 0.0006, 0.0002, 0.0008, 0.0002, 0.0037, 0.0029, -0.0026, 0.0021, 0.0001, -0.0016, -0.0002, 0.0043, -0.0001, -0.0016, 0.0006, -0.0013, -0.0041, 0.0022, -0.0013, -0.0011, -0.0008, -0.0007, -0.0013, -0.0006, -0.0006, -0.0007, -0.0, -0.0004, -0.0003, 0.0009, -0.0003, -0.0001, -0.0001, -0.0001, 0.0, -0.0001, -0.0002, 0.0, -0.0, -0.0, -0.0001, 0.0023, 0.0043])

In [74]:
regModel.intercept

-0.02854028698498677

In [75]:
trainSummary = regModel.summary
type(trainSummary)

pyspark.ml.regression.LinearRegressionTrainingSummary

In [76]:
trainSummary.meanAbsoluteError

0.029156022120266634

In [77]:
trainSummary.meanSquaredError

0.014211110364853312

In [78]:
trainSummary.r2

0.014131859677981695

In [79]:
# predykcje
regModel.transform(df_eval).show()

+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(60,[1,3,23,28,57...|0.012673119870353938|
|  0.0|(60,[0,1,2,7,23,2...| 0.03894956139590243|
|  0.0|(60,[0,1,2,3,22,3...|0.028089817612023955|
|  0.0|(60,[0,1,2,3,22,3...|0.032857002053417704|
|  0.0|(60,[0,1,2,3,22,3...| 0.03824227434040331|
|  0.0|(60,[0,1,2,15,22,...| 0.04470125804214896|
|  0.0|(60,[1,3,22,29,57...|0.013093366614811745|
|  0.0|(60,[0,1,2,3,22,3...| 0.01820400922026639|
|  0.0|(60,[0,2,8,21,24,...| 0.06421188539978266|
|  0.0|(60,[1,2,4,22,29,...| 0.00300531945764259|
|  0.0|(60,[0,1,2,4,22,2...| 0.01808183427953043|
|  0.0|(60,[1,3,22,28,57...|0.020768572297168775|
|  0.0|(60,[1,3,22,29,57...|0.014294991168959222|
|  0.0|(60,[0,1,2,9,22,2...| 0.03232506263683215|
|  0.0|(60,[1,3,22,36,57...|-0.00690761697913...|
|  0.0|(60,[0,1,2,3,22,3...|0.015960034011154214|
|  0.0|(60,[0,1,2,11,25,...| 0.01868713240699233|


#### Drzewo regresyjne

In [80]:
tree_reg = regression.DecisionTreeRegressor()

In [81]:
tree_regModel = tree_reg.fit(df_train)

In [82]:
print(tree_regModel.toDebugString)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_bd3017af2340, depth=5, numNodes=57, numFeatures=60
  If (feature 57 <= 0.016074083754750983)
   If (feature 57 <= 0.009188700621600075)
    If (feature 57 <= 0.008271828252805586)
     If (feature 5 <= 2.2628782783944144)
      If (feature 21 <= 1.1993123157773127)
       Predict: 0.0
      Else (feature 21 > 1.1993123157773127)
       Predict: 4.553734061930783E-4
     Else (feature 5 > 2.2628782783944144)
      If (feature 59 <= 0.7446789795243225)
       Predict: 0.0
      Else (feature 59 > 0.7446789795243225)
       Predict: 0.002081165452653486
    Else (feature 57 > 0.008271828252805586)
     If (feature 6 <= 2.7815593385283988)
      If (feature 28 <= 1.006510530627566)
       Predict: 0.0023059185242121443
      Else (feature 28 > 1.006510530627566)
       Predict: 0.004461221688400824
     Else (feature 6 > 2.7815593385283988)
      If (feature 0 <= 1.0107756116042783)
       Predict: 0.05357142857142857
      Else (featu

In [83]:
# predykcje
tree_regModel.transform(df_eval).show()

+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(60,[1,3,23,28,57...|0.004461221688400824|
|  0.0|(60,[0,1,2,7,23,2...|0.013018105641179112|
|  0.0|(60,[0,1,2,3,22,3...|0.005580142389840293|
|  0.0|(60,[0,1,2,3,22,3...|0.007359434795407...|
|  0.0|(60,[0,1,2,3,22,3...| 0.03278688524590164|
|  0.0|(60,[0,1,2,15,22,...|0.038523683986733566|
|  0.0|(60,[1,3,22,29,57...|0.038523683986733566|
|  0.0|(60,[0,1,2,3,22,3...|0.002305918524212...|
|  0.0|(60,[0,2,8,21,24,...| 0.04887780548628429|
|  0.0|(60,[1,2,4,22,29,...| 0.02386451116243264|
|  0.0|(60,[0,1,2,4,22,2...|0.005580142389840293|
|  0.0|(60,[1,3,22,28,57...|0.005580142389840293|
|  0.0|(60,[1,3,22,29,57...|0.005580142389840293|
|  0.0|(60,[0,1,2,9,22,2...| 0.02386451116243264|
|  0.0|(60,[1,3,22,36,57...|0.013018105641179112|
|  0.0|(60,[0,1,2,3,22,3...|0.038523683986733566|
|  0.0|(60,[0,1,2,11,25,...| 0.02386451116243264|


Las regresyjny

In [84]:
forest_reg = regression.RandomForestRegressor()

In [85]:
forest_regModel = forest_reg.fit(df_train)

In [86]:
forest_regModel.featureImportances

SparseVector(60, {0: 0.0179, 1: 0.0179, 2: 0.0119, 3: 0.0111, 4: 0.0036, 5: 0.0004, 6: 0.0012, 7: 0.028, 8: 0.0504, 9: 0.0064, 10: 0.041, 11: 0.0009, 12: 0.0019, 13: 0.0018, 15: 0.0076, 17: 0.0002, 21: 0.0373, 22: 0.006, 23: 0.0031, 24: 0.009, 25: 0.0014, 26: 0.001, 27: 0.0038, 28: 0.0106, 29: 0.0197, 30: 0.0018, 31: 0.0076, 32: 0.0013, 33: 0.0295, 34: 0.0426, 36: 0.0001, 38: 0.0027, 40: 0.0023, 41: 0.0003, 43: 0.0028, 57: 0.4933, 58: 0.0726, 59: 0.049})

In [87]:
print(forest_regModel.toDebugString)

RandomForestRegressionModel: uid=RandomForestRegressor_d23dee6cdb93, numTrees=20, numFeatures=60
  Tree 0 (weight 1.0):
    If (feature 8 <= 3.3277301298467865)
     If (feature 10 <= 3.7970696388246554)
      If (feature 7 <= 3.2408984185143432)
       If (feature 2 <= 1.0476301612375614)
        If (feature 59 <= 1.7375842855567525)
         Predict: 0.005091013880217089
        Else (feature 59 > 1.7375842855567525)
         Predict: 0.011125720643268939
       Else (feature 2 > 1.0476301612375614)
        If (feature 59 <= 0.2482263265081075)
         Predict: 0.010131446788069348
        Else (feature 59 > 0.2482263265081075)
         Predict: 0.01747900366090015
      Else (feature 7 > 3.2408984185143432)
       If (feature 57 <= 0.018346632172972638)
        If (feature 29 <= 1.0529526500814932)
         Predict: 0.015706806282722512
        Else (feature 29 > 1.0529526500814932)
         Predict: 0.05829596412556054
       Else (feature 57 > 0.018346632172972638)
        If (fe

In [88]:
# predykcje
forest_regModel.transform(df_eval).show()

+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(60,[1,3,23,28,57...|0.005470182779572...|
|  0.0|(60,[0,1,2,7,23,2...| 0.02226113535377937|
|  0.0|(60,[0,1,2,3,22,3...|0.010993702450143828|
|  0.0|(60,[0,1,2,3,22,3...| 0.01915589149791099|
|  0.0|(60,[0,1,2,3,22,3...| 0.07832606673917883|
|  0.0|(60,[0,1,2,15,22,...|0.038360998290343964|
|  0.0|(60,[1,3,22,29,57...|0.025367701018616778|
|  0.0|(60,[0,1,2,3,22,3...|0.006010905672858125|
|  0.0|(60,[0,2,8,21,24,...| 0.04343163130503726|
|  0.0|(60,[1,2,4,22,29,...|0.014655169084391656|
|  0.0|(60,[0,1,2,4,22,2...|0.012291307580146462|
|  0.0|(60,[1,3,22,28,57...|0.006866555687905394|
|  0.0|(60,[1,3,22,29,57...|0.007884591330444946|
|  0.0|(60,[0,1,2,9,22,2...|0.024677680064050335|
|  0.0|(60,[1,3,22,36,57...|0.008828451118020275|
|  0.0|(60,[0,1,2,3,22,3...|  0.0328782219650688|
|  0.0|(60,[0,1,2,11,25,...|0.015339968189898199|


Gradient-Boosted Trees regression - Podbicie Gradientowe

In [89]:
gbt_reg = regression.GBTRegressor()

In [90]:
gbt_regModel = gbt_reg.fit(df_train)

In [91]:
gbt_regModel.featureImportances

SparseVector(60, {0: 0.0185, 1: 0.0283, 2: 0.0001, 3: 0.0118, 4: 0.0039, 5: 0.0021, 6: 0.0111, 7: 0.0121, 8: 0.0152, 9: 0.0011, 10: 0.0318, 11: 0.0078, 12: 0.002, 13: 0.0019, 14: 0.0091, 15: 0.0032, 16: 0.0008, 17: 0.0004, 18: 0.0028, 19: 0.0034, 20: 0.0003, 21: 0.0336, 22: 0.0128, 23: 0.0146, 24: 0.0103, 25: 0.0115, 26: 0.0023, 27: 0.0109, 28: 0.0099, 29: 0.038, 30: 0.0326, 31: 0.0236, 32: 0.0005, 33: 0.0137, 34: 0.0092, 35: 0.0026, 36: 0.0033, 37: 0.0, 38: 0.0183, 39: 0.0088, 40: 0.01, 41: 0.0075, 42: 0.0009, 43: 0.0183, 46: 0.0238, 57: 0.2892, 58: 0.1328, 59: 0.093})

In [92]:
print(gbt_regModel.toDebugString)

GBTRegressionModel: uid=GBTRegressor_74716f0e826c, numTrees=20, numFeatures=60
  Tree 0 (weight 1.0):
    If (feature 57 <= 0.016325976814747865)
     If (feature 57 <= 0.009185485942621082)
      If (feature 57 <= 0.009183648983204513)
       If (feature 21 <= 1.1993123157773127)
        If (feature 5 <= 2.2628782783944144)
         Predict: 8.261049153242462E-5
        Else (feature 5 > 2.2628782783944144)
         Predict: 0.0015432098765432098
       Else (feature 21 > 1.1993123157773127)
        If (feature 59 <= 1.7375842855567525)
         Predict: 9.576250897773521E-4
        Else (feature 59 > 1.7375842855567525)
         Predict: 0.004601226993865031
      Else (feature 57 > 0.009183648983204513)
       If (feature 30 <= 2.037807414865838)
        If (feature 4 <= 1.6411529338454593)
         Predict: 0.004518072289156626
        Else (feature 4 > 1.6411529338454593)
         Predict: 0.0
       Else (feature 30 > 2.037807414865838)
        If (feature 0 <= 1.0107756116042783

In [93]:
# predykcje
gbt_regModel.transform(df_eval).show()

+-----+--------------------+--------------------+
|label|            features|          prediction|
+-----+--------------------+--------------------+
|  0.0|(60,[1,3,23,28,57...|0.004877856955435791|
|  0.0|(60,[0,1,2,7,23,2...|0.011228930098461865|
|  0.0|(60,[0,1,2,3,22,3...|0.005364359916471849|
|  0.0|(60,[0,1,2,3,22,3...|0.002352594307429...|
|  0.0|(60,[0,1,2,3,22,3...|0.053763175744502595|
|  0.0|(60,[0,1,2,15,22,...| 0.05429310965340222|
|  0.0|(60,[1,3,22,29,57...| 0.03815326029475324|
|  0.0|(60,[0,1,2,3,22,3...|0.004574558999340181|
|  0.0|(60,[0,2,8,21,24,...| 0.05191312059622331|
|  0.0|(60,[1,2,4,22,29,...| 0.01699322512307985|
|  0.0|(60,[0,1,2,4,22,2...|4.737875471776664E-4|
|  0.0|(60,[1,3,22,28,57...|0.004775269154383936|
|  0.0|(60,[1,3,22,29,57...|0.004167423570824612|
|  0.0|(60,[0,1,2,9,22,2...| 0.02451715940800252|
|  0.0|(60,[1,3,22,36,57...|0.009433334398780887|
|  0.0|(60,[0,1,2,3,22,3...| 0.04202362293203703|
|  0.0|(60,[0,1,2,11,25,...|0.020250475932271222|


EWALUACJA

In [96]:
from pyspark.ml import evaluation

In [102]:
evaluator_reg = evaluation.RegressionEvaluator()

In [104]:
evaluator_reg.evaluate(regModel.transform(df_eval))

0.11921036181831389

In [105]:
# rmse - drzewo
evaluator_reg.evaluate(tree_regModel.transform(df_eval))

0.11843913979184242

In [108]:
# rmse - las
evaluator_reg.evaluate(forest_regModel.transform(df_eval))

0.118299598389092

In [110]:
# rmse - gbt
evaluator_reg.evaluate(gbt_regModel.transform(df_eval))

0.11707237589294162

In [111]:
er2 = evaluation.RegressionEvaluator(metricName="r2")

In [112]:
er2.evaluate(regModel.transform(df_eval))

0.014131859677981695

In [114]:
er2.evaluate(tree_regModel.transform(df_eval))

0.026846589973356982

In [115]:
er2.evaluate(forest_regModel.transform(df_eval))

0.02913831882712037

In [116]:
er2.evaluate(gbt_regModel.transform(df_eval))

0.049176988106239716

Próba poprawy R^2

In [150]:
forest_reg2 = regression.RandomForestRegressor(numTrees=45, maxDepth=10)
er2.evaluate(forest_reg2.fit(df_train).transform(df_eval))

0.08503257393953467

Wybór najlepszych parametrów

In [151]:
from pyspark.ml import tuning
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [152]:
grid = (ParamGridBuilder()
             #.addGrid(rf.maxDepth, [2, 5, 10, 20, 30])
               .addGrid(forest_reg2.maxDepth, [2, 5, 10])
             #.addGrid(rf.maxBins, [10, 20, 40, 80, 100])
               .addGrid(forest_reg2.maxBins, [5, 10, 20])
             #.addGrid(rf.numTrees, [5, 20, 50, 100, 500])
               .addGrid(forest_reg2.numTrees, [5, 20, 50])
             .build())

In [153]:
reg_eval = evaluation.RegressionEvaluator(metricName='r2')

In [154]:
cv = tuning.CrossValidator(estimator=forest_reg2, estimatorParamMaps=grid, evaluator=reg_eval)

In [155]:
cvModel = cv.fit(df_train)

In [156]:
cvModel.avgMetrics

[0.014087437824305254,
 0.013551986687492208,
 0.01389601829786602,
 0.015278580591280325,
 0.014875765342172794,
 0.015689962074583026,
 0.015375607528121137,
 0.014852680769018061,
 0.015738197865003672,
 0.019256981657181704,
 0.019997463119295324,
 0.02015303518873287,
 0.020930631834747672,
 0.02225385546854412,
 0.02237986211768872,
 0.02050116126964674,
 0.02255097035847753,
 0.02275037149918239,
 0.011316615162475507,
 0.01744133713352499,
 0.02075022870528498,
 0.01166672442872559,
 0.02114804158712876,
 0.0220383842704995,
 0.010892321468544167,
 0.022117721421704206,
 0.02297940955259973]

Parametry najlepszego modelu

In [163]:
cvModel.bestModel._java_obj.getMaxDepth()

10

In [159]:
cvModel.bestModel._java_obj.getMaxBins()

20

In [165]:
cvModel.bestModel._java_obj.numFeatures()

60

In [166]:
cvModel.bestModel._java_obj.numTrees()

JavaObject id=o27430